# ELECTRA NLP Artifact Analysis - Training Notebook

**Professional Research Project**: Investigating dataset artifacts in NLP models

**Environment**: Google Colab with GPU (A100 recommended)

**Objective**: Train baseline ELECTRA-small model on SNLI for artifact analysis

---

## Setup & Configuration

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected. Training will be slow.")

## Mount Google Drive

Mount Drive to save checkpoints and results persistently.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set up project directory
import os
PROJECT_DIR = '/content/drive/MyDrive/electra-artifact-analysis'
os.makedirs(PROJECT_DIR, exist_ok=True)
os.makedirs(f"{PROJECT_DIR}/models", exist_ok=True)
os.makedirs(f"{PROJECT_DIR}/results", exist_ok=True)
os.makedirs(f"{PROJECT_DIR}/logs", exist_ok=True)

print(f"✓ Project directory: {PROJECT_DIR}")
print(f"✓ All subdirectories created")

## Clone Repository & Install Dependencies

In [ ]:
# Clone your repository
!git clone https://github.com/TimFrenzel/electra-nlp-artifact-analysis.git /content/electra-nlp-artifact-analysis
%cd /content/electra-nlp-artifact-analysis

In [ ]:
# Install dependencies
!pip install --upgrade pip
!pip install -q -r requirements.txt

# Verify installations
import transformers
import datasets
import evaluate

print(f"✓ Transformers: {transformers.__version__}")
print(f"✓ Datasets: {datasets.__version__}")
print(f"✓ Evaluate: {evaluate.__version__}")

## Configuration

Set hyperparameters and paths.

In [ ]:
# Training configuration
CONFIG = {
    'task': 'nli',
    'dataset': 'snli',
    'model': 'google/electra-small-discriminator',
    'num_train_epochs': 3,
    'batch_size': 32,  # Adjust based on GPU memory
    'learning_rate': 2e-5,
    'max_seq_length': 128,
    'save_steps': 500,
    'eval_steps': 500,
    'output_dir': f"{PROJECT_DIR}/models/baseline_snli",
    'seed': 42,
}

# Display configuration
import json
print("Training Configuration:")
print(json.dumps(CONFIG, indent=2))

# Save configuration
with open(f"{PROJECT_DIR}/logs/training_config.json", 'w') as f:
    json.dump(CONFIG, f, indent=2)

## Part 1: Baseline Training

Train baseline ELECTRA-small on SNLI.

**Expected Performance**: ~89% accuracy (3 epochs)

In [ ]:
# Quick test on small subset first
print("🧪 Quick test on 1000 examples...")

# Extract config values for cleaner shell command syntax
task = CONFIG['task']
dataset = CONFIG['dataset']
model = CONFIG['model']
batch_size = CONFIG['batch_size']
max_seq_length = CONFIG['max_seq_length']
seed = CONFIG['seed']

!python run.py \
    --do_train \
    --do_eval \
    --task {{task}} \
    --dataset {{dataset}} \
    --model {{model}} \
    --output_dir {{PROJECT_DIR}}/models/debug \
    --num_train_epochs 1 \
    --batch_size {{batch_size}} \
    --max_train_samples 1000 \
    --max_eval_samples 500 \
    --max_seq_length {{max_seq_length}} \
    --seed {{seed}}

print("✓ Quick test complete!")

In [ ]:
# Full training on complete dataset
print("🚀 Starting full training...")
print(f"   Dataset: {CONFIG['dataset'].upper()}")
print(f"   Model: {CONFIG['model']}")
print(f"   Epochs: {CONFIG['num_train_epochs']}")
print(f"   Expected time: 1-3 hours with A100")
print("="*60)

# Extract config values for cleaner shell command syntax
task = CONFIG['task']
dataset = CONFIG['dataset']
model = CONFIG['model']
output_dir = CONFIG['output_dir']
num_epochs = CONFIG['num_train_epochs']
batch_size = CONFIG['batch_size']
lr = CONFIG['learning_rate']
max_seq_length = CONFIG['max_seq_length']
save_steps = CONFIG['save_steps']
eval_steps = CONFIG['eval_steps']
seed = CONFIG['seed']

!python run.py \
    --do_train \
    --do_eval \
    --task {task} \
    --dataset {dataset} \
    --model {model} \
    --output_dir {output_dir} \
    --num_train_epochs {num_epochs} \
    --batch_size {batch_size} \
    --learning_rate {lr} \
    --max_seq_length {max_seq_length} \
    --save_steps {save_steps} \
    --eval_steps {eval_steps} \
    --seed {seed} \
    --fp16

print("\n✓ Training complete!")

## Evaluate Baseline Performance

In [ ]:
# Evaluate on full validation set
task = CONFIG['task']
dataset = CONFIG['dataset']
output_dir = CONFIG['output_dir']
batch_size = CONFIG['batch_size']

!python run.py \
    --do_eval \
    --task {task} \
    --dataset {dataset} \
    --model {output_dir} \
    --output_dir {PROJECT_DIR}/results/baseline \
    --batch_size {batch_size}

# Load and display results
import json

with open(f"{PROJECT_DIR}/results/baseline/eval_results.json", 'r') as f:
    results = json.load(f)

print("\n" + "="*60)
print("BASELINE PERFORMANCE")
print("="*60)
for key, value in results.items():
    if isinstance(value, float):
        print(f"{key:30s}: {value:.4f}")
    else:
        print(f"{key:30s}: {value}")
print("="*60)

# Check against expected performance
if 'eval_accuracy' in results:
    acc = results['eval_accuracy']
    if acc >= 0.89:
        print(f"✓ EXCELLENT: Accuracy {acc:.2%} meets/exceeds expected ~89%")
    elif acc >= 0.85:
        print(f"✓ GOOD: Accuracy {acc:.2%} is reasonable (target: ~89%)")
    else:
        print(f"⚠️ WARNING: Accuracy {acc:.2%} below expected ~89%. Check training.")

## Save Training Summary

In [ ]:
# Create training summary
import datetime

summary = {
    'timestamp': datetime.datetime.now().isoformat(),
    'config': CONFIG,
    'results': results,
    'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU',
    'model_path': CONFIG['output_dir'],
    'notes': [
        'Baseline training complete',
        'Ready for artifact analysis (Part 1)',
        'Checkpoints saved to Google Drive'
    ]
}

summary_path = f"{PROJECT_DIR}/logs/training_summary.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✓ Training summary saved to: {summary_path}")
print("\n📋 Next Steps:")
print("   1. Run analysis notebook (Part 1: Artifact Analysis)")
print("   2. Identify dataset artifacts and spurious correlations")
print("   3. Design mitigation strategy (Part 2)")

## Optional: Quick Artifact Check

Quick hypothesis-only baseline to check for artifacts.

In [ ]:
# This is a preview - full analysis in separate notebook
from analysis.error_analysis import ErrorAnalyzer

print("🔍 Quick artifact check (hypothesis-only baseline)...")
print("   Random baseline: 33.3% (3-class classification)")
print("   Biased baseline: ~67% (indicates artifacts)")
print("   Full model: ~89%\n")

analyzer = ErrorAnalyzer(CONFIG['output_dir'])
hyp_results = analyzer.analyze_hypothesis_only(max_samples=1000)

hyp_acc = hyp_results['hypothesis_only_accuracy']
print(f"\nHypothesis-only accuracy: {hyp_acc:.2%}")

if hyp_acc > 0.6:
    print("⚠️ HIGH ARTIFACT SIGNAL: Model likely exploits hypothesis-only biases")
    print("   → This confirms need for debiasing (Part 2)")
elif hyp_acc > 0.5:
    print("⚠️ MODERATE ARTIFACT SIGNAL: Some hypothesis-only bias present")
else:
    print("✓ LOW ARTIFACT SIGNAL: Model relies on full context")

print("\nFor full artifact analysis, see: analysis_part1.ipynb")

---

## Training Complete ✓

### Files Saved to Google Drive:
- Model checkpoints: `{PROJECT_DIR}/models/baseline_snli/`
- Evaluation results: `{PROJECT_DIR}/results/baseline/`
- Training logs: `{PROJECT_DIR}/logs/`

### Next Notebooks:
1. **analysis_part1.ipynb** - Artifact analysis and error characterization
2. **mitigation_part2.ipynb** - Implement debiasing methods

### For Report:
- Baseline accuracy: Record in Part 1
- Training details: Include in Method section
- Hypothesis-only baseline: Include in Analysis section